# Restructuring Data in Pandas

This notebook is inspired from the textbook:

[Jake VanderPlas: Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/index.html)

In [1]:
import pandas as pd
import numpy as np

## Agenda

- How to read data with different encodings
- How to find missing values in dataframes
- Dealing with missing values
- Combining dataframes
- Groupby method in Pandas
- Aggregation with groupby

## Reading Data - Encodings

[Download Data](https://www.kaggle.com/kwullum/fatal-police-shootings-in-the-us?select=PoliceKillingsUS.csv)

[Washington Post Repo](https://github.com/washingtonpost/data-police-shootings)

In [13]:
df = pd.read_csv('data/datasets_2647_4395_PoliceKillingsUS.csv',encoding='ascii')

UnicodeDecodeError: 'ascii' codec can't decode byte 0x96 in position 25272: ordinal not in range(128)

### Encodings

In [4]:
before = "This is the euro symbol: €"

In [5]:
after = before.encode()

after

b'This is the euro symbol: \xe2\x82\xac'

In [17]:
## try decoding with utf-8
## try decoding with ascii

## encode with ascii make sure that setting errors = 'replace'
## decoce with using ascii

In [9]:
    after.decode('ascii',errors='replace')

'This is the euro symbol: ���'

### Solution to the encoding-decoding mess chardet library

[Chardet package](https://pypi.org/project/chardet/)

In [10]:
import sys
!{sys.executable} -m pip install chardet

[For more on installing packages from notebook](https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/)

In [11]:
import chardet
# look at the first ten thousand bytes to guess the character encoding
with open("data/datasets_2647_4395_PoliceKillingsUS.csv", 'rb') as rawdata:
    result = chardet.detect(rawdata.read(2000))

# check what the character encoding might be
print(result)

{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}


## try encoding = 'ascii'
## read the error carefully did it changed?


You'll get ascii with 100% confidence.  You can then raise the number of characters being read, which then gives a new suggestion that does work


- [Kaggle - Dealing with Char Encodings](https://www.kaggle.com/rtatman/data-cleaning-challenge-character-encodings)

- [Python Standard Encodings](https://docs.python.org/3/library/codecs.html#standard-encodings)

In [14]:
df = pd.read_csv('data/datasets_2647_4395_PoliceKillingsUS.csv', encoding='Windows-1252')

In [22]:
print(df.shape)
df.head()

(2535, 14)


,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera
0,3,Tim Elliot,02/01/15,shot,gun,53.0,M,A,Shelton,WA,True,attack,Not fleeing,False
1,4,Lewis Lee Lembke,02/01/15,shot,gun,47.0,M,W,Aloha,OR,False,attack,Not fleeing,False
2,5,John Paul Quintero,03/01/15,shot and Tasered,unarmed,23.0,M,H,Wichita,KS,False,other,Not fleeing,False
3,8,Matthew Hoffman,04/01/15,shot,toy weapon,32.0,M,W,San Francisco,CA,True,attack,Not fleeing,False
4,9,Michael Rodriguez,04/01/15,shot,nail gun,39.0,M,H,Evans,CO,False,attack,Not fleeing,False


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2535 entries, 0 to 2534
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       2535 non-null   int64  
 1   name                     2535 non-null   object 
 2   date                     2535 non-null   object 
 3   manner_of_death          2535 non-null   object 
 4   armed                    2526 non-null   object 
 5   age                      2458 non-null   float64
 6   gender                   2535 non-null   object 
 7   race                     2340 non-null   object 
 8   city                     2535 non-null   object 
 9   state                    2535 non-null   object 
 10  signs_of_mental_illness  2535 non-null   bool   
 11  threat_level             2535 non-null   object 
 12  flee                     2470 non-null   object 
 13  body_camera              2535 non-null   bool   
dtypes: bool(2), float64(1), 

### Detecting Null Values

In [26]:
## find null values in each column

df.isna().sum()

id                           0
name                         0
date                         0
manner_of_death              0
armed                        9
age                         77
gender                       0
race                       195
city                         0
state                        0
signs_of_mental_illness      0
threat_level                 0
flee                        65
body_camera                  0
dtype: int64

[Difference between `isna` and  `isnull`](https://datascience.stackexchange.com/questions/37878/difference-between-isna-and-isnull-in-pandas)

## Dealing with Missing Values

### Droping Rows with NaN values

In [33]:
#Playing:
df.race.value_counts()

W    1201
B     618
H     423
A      39
N      31
O      28
Name: race, dtype: int64

In [32]:
df.dropna(subset=['age','race'],how='any')

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera
0,3,Tim Elliot,02/01/15,shot,gun,53.0,M,A,Shelton,WA,True,attack,Not fleeing,False
1,4,Lewis Lee Lembke,02/01/15,shot,gun,47.0,M,W,Aloha,OR,False,attack,Not fleeing,False
2,5,John Paul Quintero,03/01/15,shot and Tasered,unarmed,23.0,M,H,Wichita,KS,False,other,Not fleeing,False
3,8,Matthew Hoffman,04/01/15,shot,toy weapon,32.0,M,W,San Francisco,CA,True,attack,Not fleeing,False
4,9,Michael Rodriguez,04/01/15,shot,nail gun,39.0,M,H,Evans,CO,False,attack,Not fleeing,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2519,2805,Aries Clark,25/07/17,shot,undetermined,16.0,M,B,Marion,AR,False,undetermined,Not fleeing,False
2523,2808,Kesharn K. Burney,26/07/17,shot,vehicle,25.0,M,B,Dayton,OH,False,attack,Car,False
2525,2820,Deltra Henderson,27/07/17,shot,gun,39.0,M,B,Homer,LA,False,attack,Car,False
2533,2817,Isaiah Tucker,31/07/17,shot,vehicle,28.0,M,B,Oshkosh,WI,False,attack,Car,True


In [26]:
## use of dropna with default

In [29]:
df.dropna(thresh=2)

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera
0,3,Tim Elliot,02/01/15,shot,gun,53.0,M,A,Shelton,WA,True,attack,Not fleeing,False
1,4,Lewis Lee Lembke,02/01/15,shot,gun,47.0,M,W,Aloha,OR,False,attack,Not fleeing,False
2,5,John Paul Quintero,03/01/15,shot and Tasered,unarmed,23.0,M,H,Wichita,KS,False,other,Not fleeing,False
3,8,Matthew Hoffman,04/01/15,shot,toy weapon,32.0,M,W,San Francisco,CA,True,attack,Not fleeing,False
4,9,Michael Rodriguez,04/01/15,shot,nail gun,39.0,M,H,Evans,CO,False,attack,Not fleeing,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2530,2822,Rodney E. Jacobs,28/07/17,shot,gun,31.0,M,NaN,Kansas City,MO,False,attack,Not fleeing,False
2531,2813,TK TK,28/07/17,shot,vehicle,NaN,M,NaN,Albuquerque,NM,False,attack,Car,False
2532,2818,Dennis W. Robinson,29/07/17,shot,gun,48.0,M,NaN,Melba,ID,False,attack,Car,False
2533,2817,Isaiah Tucker,31/07/17,shot,vehicle,28.0,M,B,Oshkosh,WI,False,attack,Car,True


In [29]:
## how to drop only rows?

## how to drop columns?

## how about the other parameters: how, thresh, inplace, subset

### Filling Null Values

In [30]:
## dropping might not be the best practice


In [38]:
#replace missing values in age column with mean age value

mean_age=int(df.age.mean())

df.age.fillna(value=mean_age,inplace=True)
df.age.isna().sum()

0

In [46]:
df.race.value_counts(normalize=True)

W    0.513248
B    0.264103
H    0.180769
A    0.016667
N    0.013248
O    0.011966
Name: race, dtype: float64

In [48]:
df.city.value_counts()

Los Angeles    39
Phoenix        31
Houston        27
Chicago        25
Las Vegas      21
               ..
Kenner          1
Snohomish       1
Fairfax         1
Brockton        1
Milton          1
Name: city, Length: 1417, dtype: int64

## `Apply`, `Map` and `ApplyMap`

[Dataset](https://www.census.gov/data/tables/time-series/demo/popest/2010s-total-cities-and-towns.html)

In [31]:
## reading data from excel sheet

In [63]:
census = pd.read_excel('data/SUB-IP-EST2019-ANNRNK.xlsx',skiprows=3,skipfooter=5)

census.tail(9)

,Unnamed: 0,Unnamed: 1,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
779,780,"Texas City city, Texas",45099,45084,45284,45179,45577,45900,46383,47299,47994,48417,49078,50094
780,781,"Cerritos city, California",49041,49043,49042,49223,49384,49507,49532,49546,50346,50697,50288,49859
781,782,"Coral Gables city, Florida",46780,46746,46963,47613,49049,49762,50176,50275,50396,50628,50125,49700
782,783,"Enid city, Oklahoma",49379,49385,49530,49529,50018,50787,51113,51478,50869,50132,49723,49688
783,784,"Lakewood city, Ohio",52131,52131,52026,51662,51400,51257,51088,50811,50528,50211,50012,49678
784,785,"Troy city, New York",50129,50162,50081,50020,49848,49921,49968,49776,49597,49433,49354,49154
785,786,"Saginaw city, Michigan",51508,51469,51363,51012,50728,50272,49798,49230,48902,48696,48310,48115
786,787,"Niagara Falls city, New York",50193,50031,49996,49760,49526,49328,49083,48769,48526,48249,48012,47720
787,788,"Charleston city, West Virginia",51400,51340,51274,50992,50858,50657,50133,49457,48789,48040,47172,46536


In [33]:
## use default? Is it ok?

## use skiprows. Are we done?

In [73]:
## dropping columns
try:
    census.drop(columns=['Unnamed: 0'],inplace=True,axis='columns')
except:
    None
## renaming columns
census.rename(columns={'Unnamed: 1':'Place'},inplace=True)



### Apply Method

In [97]:
###Apply method owrks with dataframes and not with series.  

###Apply takes columns or rows in as arguments.  

census['Place']


census.apply(lambda x: x['Place'])

KeyError: 'Place'

In [90]:
census[[2010,2011, 2012, 2013, 2014, 2015]].apply(lambda x: x.std(),axis=1)

0      103351.545722
1       54149.243158
2       11799.820090
3       70736.223456
4       51227.546397
           ...      
783       429.226747
784       111.958326
785       794.629914
786       448.081540
787       662.351996
Length: 788, dtype: float64

### Applymap Method

In [82]:
## try applymap as above
census[[2010,2011, 2012, 2013, 2014, 2015]].applymap(lambda x:x/1000)

,2010,2011,2012,2013,2014,2015
0,8190.209,8272.948,8346.693,8396.091,8433.806,8463.049
1,3795.512,3820.876,3851.202,3881.622,3909.901,3938.568
2,2697.477,2708.114,2719.141,2725.731,2727.066,2724.344
3,2100.280,2126.032,2161.593,2199.391,2241.826,2286.908
4,1449.038,1469.796,1499.274,1526.491,1555.445,1583.690
...,...,...,...,...,...,...
783,52.026,51.662,51.400,51.257,51.088,50.811
784,50.081,50.020,49.848,49.921,49.968,49.776
785,51.363,51.012,50.728,50.272,49.798,49.230
786,49.996,49.760,49.526,49.328,49.083,48.769


In [ ]:
## Splitting cities and states 

In [143]:
census['city']=census.Place.map(lambda s: s.replace(' city','').split(",")[0].strip())
census['state']=census.Place.map(lambda s: s.replace(' city','').split(",")[-1].strip())

In [144]:
## creating the abbrevated state names

In [145]:
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

census['state'].map(lambda s: states[s])

KeyError: 'New York'

'New York'

In [146]:
states = {values: key for key, values in states.items()}

In [147]:
census['abbr'] = census.state.map(lambda state: states[state])

In [148]:
census.head(2)

,Place,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,city,state,abbr
0,"New York city, New York",8175133,8175031,8190209,8272948,8346693,8396091,8433806,8463049,8469153,8437478,8390081,8336817,New York,New York,NY
1,"Los Angeles city, California",3792621,3793139,3795512,3820876,3851202,3881622,3909901,3938568,3963226,3975788,3977596,3979576,Los Angeles,California,CA


[More on the differences between `apply`,`map` and `applymap`](https://stackoverflow.com/questions/19798153/difference-between-map-applymap-and-apply-methods-in-pandas)

## Merging Datasets

This Part of the notebook is adopted from:

[PythonDataScienceHandbook - Merge -Join](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html)

In [1]:
import pandas as pd
import numpy as np

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

In [2]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
display('df1', 'df2')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


In [152]:


df3 = pd.merge(df1, df2)
display('df1', 'df2','pd.merge(df1, df2)')



df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df2
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014

pd.merge(df1, df2)
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

In [154]:


df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
display('df3', 'df4', 'pd.merge(df3, df4)')



df3
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

df4
         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve

pd.merge(df3, df4)
  employee        group  hire_date supervisor
0      Bob   Accounting       2008      Carly
1     Jake  Engineering       2012      Guido
2     Lisa  Engineering       2004      Guido
3      Sue           HR       2014      Steve

In [155]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})
display('df1', 'df5', "pd.merge(df1, df5)")

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets


In [178]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})
display('df1', 'df3', 'pd.merge(df1, df3, left_on="employee", right_on="name")')

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df3
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

pd.merge(df1, df3, left_on="employee", right_on="name")
  employee        group  name  salary
0      Bob   Accounting   Bob   70000
1     Jake  Engineering  Jake   80000
2     Lisa  Engineering  Lisa  120000
3      Sue           HR   Sue   90000

For more details please check this excellent notebook:

[Python Data Science Handbook - Merge-Join](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html)

And the `pandas` documentation

[Pandas - Merging](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html)

In [184]:
df_merged = pd.merge(census[['city', 'state', 2016]], df, on = 'city', how = 'inner',suffixes=['_census','_police'])

In [185]:
df_merged.head()

,city,state_census,2016,id,name,date,manner_of_death,armed,age,gender,race,state_police,signs_of_mental_illness,threat_level,flee,body_camera
0,New York,New York,8469153,411,David Felix,25/04/15,shot,unarmed,24.0,M,B,NY,True,attack,Foot,False
1,New York,New York,8469153,1470,George Tillman,17/04/16,shot,gun,32.0,M,B,NY,False,attack,Foot,False
2,New York,New York,8469153,1543,Garry Conrad,18/05/16,shot,knife,46.0,M,W,NY,True,other,Not fleeing,False
3,New York,New York,8469153,1631,Rashaun Lloyd,19/06/16,shot,gun,25.0,M,B,NY,False,attack,Not fleeing,False
4,New York,New York,8469153,1706,Orville Edwards,11/07/16,shot,gun,39.0,M,B,NY,False,attack,Foot,False


In [225]:
df_merged.state_census.value_counts().loc['California']

341

In [228]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1527 entries, 0 to 1526
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   city                     1527 non-null   object 
 1   state_census             1527 non-null   object 
 2   2016                     1527 non-null   int64  
 3   id                       1527 non-null   int64  
 4   name                     1527 non-null   object 
 5   date                     1527 non-null   object 
 6   manner_of_death          1527 non-null   object 
 7   armed                    1525 non-null   object 
 8   age                      1527 non-null   float64
 9   gender                   1527 non-null   object 
 10  race                     1419 non-null   object 
 11  state_police             1527 non-null   object 
 12  signs_of_mental_illness  1527 non-null   bool   
 13  threat_level             1527 non-null   object 
 14  flee                    

In [222]:
df_merged.state_census

0         New York
1         New York
2         New York
3         New York
4         New York
           ...    
1522      Oklahoma
1523    California
1524    California
1525      Michigan
1526      Michigan
Name: state_census, Length: 1527, dtype: object

In [204]:
df_merged.apply(lambda x: x.loc[2016],axis="columns")

0       8469153
1       8469153
2       8469153
3       8469153
4       8469153
         ...   
1522      49769
1523      50346
1524      50346
1525      48902
1526      48902
Length: 1527, dtype: int64

## Groupby

In [177]:
grouped = df_merged.groupby(by = 'city')
grouped.get_group('New York').race.value_counts()

NameError: name 'df_merged' is not defined

In [168]:
grouped[2016].mean()

NameError: name 'grouped' is not defined

In [169]:
grouped.aggregate(['min', np.median, max])['age']

NameError: name 'grouped' is not defined

In [170]:
my_func = lambda x: x.max() - x.min()

In [171]:
grouped.aggregate({'date': 'max', 'age': 'min', 2016: my_func})

NameError: name 'grouped' is not defined

In [172]:
grouped = df_merged.groupby(by = ['city', 'state_x'])


NameError: name 'df_merged' is not defined

In [173]:
grouped.get_group(('New York', 'New York')).race.value_counts()

NameError: name 'grouped' is not defined

In [174]:
grouped[2016].mean()

NameError: name 'grouped' is not defined

In [175]:
(grouped['city'].count()/grouped[2016].mean()).sort_values(ascending =False)

NameError: name 'grouped' is not defined

In [176]:
grouped.get_group('Toledo')

NameError: name 'grouped' is not defined

## Resources

[More on encodings](https://realpython.com/python-encodings-guide/)

In [7]:
dfa = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue','Lisa'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR','Cat'],'other':['Bob','Jake','Jake','Sue','Steve']})

In [10]:
(dfa.employee).concat(other)

AttributeError: 'Series' object has no attribute 'concat'

In [15]:
pd.concat([dfa.employee,dfa.other]).unique()

array(['Bob', 'Jake', 'Lisa', 'Sue', 'Steve'], dtype=object)

In [19]:
np.unique(dfa[['employee','other']])

array(['Bob', 'Jake', 'Lisa', 'Steve', 'Sue'], dtype=object)